In [1]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2,f_classif


In [2]:
import numpy as np
import  pandas as pd
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier


cd=pd.read_csv("telecom.csv")
cdf=pd.DataFrame(cd)
list1 = [ 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                'TechSupport','StreamingTV', 'StreamingMovies']
for i in list1:
    cdf[i] = cdf[i].replace({'No internet service':'No'})
cdf['MultipleLines'] = cdf['MultipleLines'].replace({'No phone service':'No'})

list2=[ 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection','Churn',
                'TechSupport','StreamingTV', 'StreamingMovies','MultipleLines','Partner','Dependents','PaperlessBilling','PhoneService']
for i in list2:
    cdf[i]= cdf[i].replace({'Yes':1,'No':0})
gen=pd.get_dummies(cdf["gender"],drop_first = True)
cdf=pd.concat((cdf,gen),axis=1)
cdf.drop(['gender'],axis=1,inplace=True)
y=pd.get_dummies(cdf["InternetService"])
y.drop(['No'],axis=1,inplace=True)
cdf.drop(['InternetService'],axis=1,inplace=True)
cdf=pd.concat((cdf,y),axis=1)
x=pd.get_dummies(cdf["Contract"])
cdf.drop(["Contract"],axis=1,inplace=True)
cdf=pd.concat((cdf,x),axis=1)
cdf.drop(["PaymentMethod"],axis=1,inplace=True)
cdf.drop(["customerID"],axis=1,inplace=True)
cdf['tenure']=cdf['tenure'].replace({0:0.1})
ten=np.log(cdf['tenure'])
cdf.drop(["tenure"],axis=1,inplace=True)
cdf=pd.concat((cdf,ten),axis=1)
cdf['TotalCharges'] =pd.to_numeric(cdf['TotalCharges'], errors ='coerce')
z=cdf.dropna()
index =cdf['TotalCharges'].index[cdf['TotalCharges'].apply(np.isnan)]
c=[]
c.extend(index)

d=cdf.loc[c,["MonthlyCharges"]]


x= z["MonthlyCharges"].values

m=len(x)
x=x.reshape((m,-1))




y= z["TotalCharges"].values
y = y.reshape((m,-1))

regmodel=linear_model.LinearRegression()
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.3,random_state=42)
regmodel.fit(x_train,y_train)
a= regmodel.predict(d)

list1=[]
for i in a:
    list1.extend(i)

r2= regmodel.score(x,y)
print("Accuracy of linear model used for predicting a null values of TotalCharges in the data set : ", r2)

for i in list1:
    cdf['TotalCharges']=(cdf['TotalCharges'].fillna(i,limit= 1))


tc=np.log(cdf['TotalCharges'])
cdf.drop(['TotalCharges'],axis=1,inplace=True)
cdf=pd.concat((cdf,tc),axis=1)
mc=np.log(cdf['MonthlyCharges'])
cdf.drop(['MonthlyCharges'],axis=1,inplace=True)
cdf=pd.concat((cdf,mc),axis=1)

cdf=cdf.abs()



Accuracy of linear model used for predicting a null values of TotalCharges in the data set :  0.4238558780398537


In [26]:
x=cdf.drop('Churn',axis=1)
y=cdf['Churn']
list_a=[]
list_f=[]
list1=[f_classif,chi2]
for w in list1:
    
    print("The feature selection based on", w," Method:")
    print()
    for i in range(1,22):
        print("No. of features selected:   ",i )
        print()
        bestfeatures=SelectKBest(score_func=w,k=i)
        fit=bestfeatures.fit_transform(x,y)
        col1=x.columns.values[bestfeatures.get_support()]
        scores=bestfeatures.scores_[bestfeatures.get_support()]
        print("The name of the columns corresponding to the scores:")
        print()
        name_scores=list(zip(col1,scores))
        print(name_scores)
        x1=cdf[list(col1)]
        x_train,x_test,y_train,y_test= train_test_split(x1,y,test_size=0.3,random_state=1)
        kfold = model_selection.KFold(n_splits=10, random_state =42)
        logmodel= LogisticRegression(solver='lbfgs', multi_class='auto')
        logmodel.fit(x_train,y_train)
        pre=logmodel.predict(x_test)
        print()
        print("Performance of the different models with",i,"th","best columns of the dataset: ")
        print()
        print("Accuracy of the logistic model:  " , accuracy_score(y_test, pre)*100)
        print("F1_score of the model: ", f1_score(y_test,pre)*100)
        list_a.append(accuracy_score(y_test, pre))
        list_f.append(f1_score(y_test,pre))
        res=confusion_matrix(y_test,pre)
        print(res)
        print()
        print()

#decision tree

        classifier_entropy = DecisionTreeClassifier(criterion='entropy', random_state=42, max_depth = 20 )
        classifier_entropy.fit(x_train,y_train)
        y_ped=classifier_entropy.predict(x_test)
        print("Accuracy of Decision tree model: ",accuracy_score(y_test,y_ped)*100)
        print("F1_score of Decision tree model: ", f1_score(y_test,y_ped)*100)
        print(confusion_matrix(y_test,y_ped))
        print()
        print()
#KNN
        classifier=KNeighborsClassifier(n_neighbors=83,metric='euclidean')
        classifier.fit(x_train,y_train)
        y_pred =classifier.predict(x_test)
        print("Accuracy of KNN model: ",accuracy_score(y_test,y_pred)*100)
        print("F1_score of KNN model: ", f1_score(y_test,y_pred)*100)
        print(confusion_matrix(y_test,y_pred))
        print()
 #NB
        print()
        label=list(y_train)
        l=x_train
        model=GaussianNB()
        model.fit(l,label)
        predicted=model.predict(x_test) 
        print(" Accuracy of Naive byes model: ",accuracy_score(y_test,predicted)*100)
        print("F1 score of Naive byes model:  ",f1_score(y_test,predicted)*100)
        print(confusion_matrix(y_test,predicted))
    
          





The feature selection based on <function f_classif at 0x0000015AA1F5AD38>  Method:

No. of features selected:    1

The name of the columns corresponding to the scores:

[('Month-to-month', 1382.340696976842)]

Performance of the different models with 1 th best columns of the dataset: 

Accuracy of the logistic model:   75.01183151916706
F1_score of the model:  0.0
[[1585    0]
 [ 528    0]]


Accuracy of Decision tree model:  75.01183151916706
F1_score of Decision tree model:  0.0
[[1585    0]
 [ 528    0]]


Accuracy of KNN model:  75.01183151916706
F1_score of KNN model:  0.0
[[1585    0]
 [ 528    0]]


 Accuracy of Naive byes model:  65.97255087553242
F1 score of Naive byes model:   57.07462686567164
[[916 669]
 [ 50 478]]
No. of features selected:    2

The name of the columns corresponding to the scores:

[('Month-to-month', 1382.340696976842), ('tenure', 1162.7579488308727)]

Performance of the different models with 2 th best columns of the dataset: 



C:\Users\Slok\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Slok\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Slok\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Accuracy of the logistic model:   77.04685281590156
F1_score of the model:  41.91616766467065
[[1453  132]
 [ 353  175]]


Accuracy of Decision tree model:  76.28963558920965
F1_score of Decision tree model:  42.74285714285715
[[1425  160]
 [ 341  187]]


Accuracy of KNN model:  76.52626597255087
F1_score of KNN model:  40.24096385542168
[[1450  135]
 [ 361  167]]


 Accuracy of Naive byes model:  75.9110269758637
F1 score of Naive byes model:   53.25987144168962
[[1314  271]
 [ 238  290]]
No. of features selected:    3

The name of the columns corresponding to the scores:

[('Fiber optic', 738.0460424544476), ('Month-to-month', 1382.340696976842), ('tenure', 1162.7579488308727)]

Performance of the different models with 3 th best columns of the dataset: 

Accuracy of the logistic model:   80.07572172266919
F1_score of the model:  51.66475315729048
[[1467  118]
 [ 303  225]]


Accuracy of Decision tree model:  78.75059157595835
F1_score of Decision tree model:  56.534365924491766
[[137

C:\Users\Slok\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Accuracy of KNN model:  80.21769995267393
F1_score of KNN model:  58.36653386454182
[[1402  183]
 [ 235  293]]


 Accuracy of Naive byes model:  72.50354945575012
F1 score of Naive byes model:   60.76975016880487
[[1082  503]
 [  78  450]]
No. of features selected:    11

The name of the columns corresponding to the scores:

[('Dependents', 195.1493137732415), ('OnlineSecurity', 212.66619940319887), ('TechSupport', 196.25540507248175), ('PaperlessBilling', 268.9852180928093), ('Fiber optic', 738.0460424544476), ('Month-to-month', 1382.340696976842), ('One year', 229.90574088120178), ('Two year', 707.9192540580779), ('tenure', 1162.7579488308727), ('TotalCharges', 434.5374002950308), ('MonthlyCharges', 325.8897865399093)]

Performance of the different models with 11 th best columns of the dataset: 

Accuracy of the logistic model:   80.26502602934217
F1_score of the model:  57.230769230769226
[[1417  168]
 [ 249  279]]


Accuracy of Decision tree model:  74.34926644581165
F1_score of De

...

Accuracy of KNN model:  80.8329389493611
F1_score of KNN model:  59.621136590229305
[[1409  176]
 [ 229  299]]


 Accuracy of Naive byes model:  73.78135352579271
F1 score of Naive byes model:   61.42061281337048
[[1118  467]
 [  87  441]]
No. of features selected:    20

The name of the columns corresponding to the scores:

[('SeniorCitizen', 134.35154479888715), ('Partner', 82.41208263843043), ('Dependents', 133.03644287868082), ('MultipleLines', 6.548511590728465), ('OnlineSecurity', 147.29585790556192), ('OnlineBackup', 31.217694023039066), ('DeviceProtection', 20.226662195984314), ('TechSupport', 135.55978268636352), ('StreamingTV', 17.334234804462824), ('StreamingMovies', 16.242530716789197), ('PaperlessBilling', 105.68086299962546), ('Male', 0.2586986175941518), ('DSL', 71.31318025219977), ('Fiber optic', 374.4762164498734), ('Month-to-month', 519.8953106092886), ('One year', 176.12317121760617), ('Two year', 488.578090256898), ('tenure', 599.9539968592322), ('TotalCharges', 142

C:\Users\Slok\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Accuracy of KNN model:  80.92759110269758
F1_score of KNN model:  60.059464816650156
[[1407  178]
 [ 225  303]]


 Accuracy of Naive byes model:  73.82867960246095
F1 score of Naive byes model:   61.46341463414634
[[1119  466]
 [  87  441]]
No. of features selected:    21

The name of the columns corresponding to the scores:

[('SeniorCitizen', 134.35154479888715), ('Partner', 82.41208263843043), ('Dependents', 133.03644287868082), ('PhoneService', 0.09726062494293952), ('MultipleLines', 6.548511590728465), ('OnlineSecurity', 147.29585790556192), ('OnlineBackup', 31.217694023039066), ('DeviceProtection', 20.226662195984314), ('TechSupport', 135.55978268636352), ('StreamingTV', 17.334234804462824), ('StreamingMovies', 16.242530716789197), ('PaperlessBilling', 105.68086299962546), ('Male', 0.2586986175941518), ('DSL', 71.31318025219977), ('Fiber optic', 374.4762164498734), ('Month-to-month', 519.8953106092886), ('One year', 176.12317121760617), ('Two year', 488.578090256898), ('tenure', 

C:\Users\Slok\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Accuracy of KNN model:  80.88026502602933
F1_score of KNN model:  60.15779092702169
[[1404  181]
 [ 223  305]]


 Accuracy of Naive byes model:  74.06530998580217
F1 score of Naive byes model:   61.678321678321666
[[1124  461]
 [  87  441]]
